Автоматическое дифференцирование с ``torch.autograd``
=======================================


При обучении нейронных сетей чаще всего используется метод **Обратного распространения ошибки**. В данном алгоритме параметры (веса) настраиваются в соответствие с **градиентом** функции потерь по заданному параметру. 

Для расчета градиентов функции ошибки PyTorch имеет встроенный механизм дифференциации под названием ``torch.autograd``. Механизм поддерживает автоматическое вычисление градиента для любого вычисленного графа.

Рассмотрим простейшую однослойную нейронную сеть с входными данными ``x``, параметрами ``w`` и ``b``, а также с некоторой функцией потерь. Данную нейронную сеть в PyTorch можно реализовать следующим образом:

In [4]:
import torch

In [5]:
x = torch.ones(5) # входные данные
y = torch.zeros(3) # ожидаемые выходные данные
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b # линейная модель, прогноз
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [6]:
print(f"Loss: {loss}")

Loss: 0.6736685633659363


Тензоры, функции и вычислительный граф
------------------------------------------

Код, представленный выше, определяет следующий **computational graph**:

<img src="images/comp-graph.png" alt="image info" />

В этой сети параметры ``w`` и ``b`` нужно оптимизировать. Таким образом, нам нужно иметь возможность вычислять градиенты функции потерь по отношению к этим переменным. Для этого мы и устанавливаем ``requires_grad=True`` конкретно у ЭТИХ ТЕНЗОРОВ.

<div class="alert alert-info"><h4>Примечание</h4><p>Вы можете установить значение ``requires_grad`` при создании тензора или позже, используя метод ``x.requires_grad_(True)``.</p></div>

Функция, которую мы применяем к тензорам для построения вычислительного графа, на самом деле является объектом класса ``Function``.

Этот объект знает, как вычислить функцию в направлении *forward*, а также как вычислить ее производную на шаге *backward propagation* (обратное распространение ошибки).

Ссылка на функцию "обратного распространения ошибки" хранится в свойстве ``grad_fn`` тензора. 

Дополнительную информацию о ``Function`` можно найти в https://pytorch.org/docs/stable/autograd.html#function.

In [7]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x000001C93D6FE580>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x000001C93D6405E0>


Вычисление градиентов
-------------------

Для оптимизации весов нам нужно вычислить производные нашей функции потерь по ``w`` и ``b``, а именно нам нужны

$$\frac{\partial loss}{\partial w}$$

$$\frac{\partial loss}{\partial b}$$

при некоторых фиксированных значениях ``x`` и ``y``. 

Чтобы вычислить эти производные, нужно вызвать ``loss.backward()``, а затем извлечь значения из ``w.grad`` и ``b.grad``:

In [8]:
loss.backward()

In [9]:
print(w.grad)
print(b.grad)

tensor([[0.0883, 0.2289, 0.1415],
        [0.0883, 0.2289, 0.1415],
        [0.0883, 0.2289, 0.1415],
        [0.0883, 0.2289, 0.1415],
        [0.0883, 0.2289, 0.1415]])
tensor([0.0883, 0.2289, 0.1415])


<div class="alert alert-info"><h4>Примечание</h4><p>
1. Мы можем получить свойства ``grad`` только для листовых узлов вычислительного графа, у которых свойство ``requires_grad`` установлено в ``True``. Для всех остальных узлов нашего графа градиенты будут недоступны.<br>2. Мы можем выполнять вычисления градиента тоьлко один раз в обратном направлении на заданном графе из соображений производительности. Если нам нужно сделать несколько ``backward` колов на одном и том же графе, то в таком случае необходимо передать ``retain_graph=True`` при вызове ``backward``.
</p></div>

Отключение трекинг градиента (disable gradient tracking)
---------------------------

По-умолчанию все тензоры с параметром ``requires_grad=True`` отслеживают историю своих вычислений и поддерживают вычисление градиента. Однако, бывают случаи, когда нам этого делать не нужно. Например, когда мы обучили модель и просто хотим применить ее к каким-то входным данным, т.е. мы хотим делать только прямые (forward) вычисления по сетке. Мы можем остановить отслеживание вычислений, окружив наш вычислитеьлный код блоком ``torch.no_grad()``.

In [10]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

True


In [11]:
with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

False


Другой способ добиться того же результата - применить ``detach()`` метод на тензор:

In [12]:
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


Причины для отключения **gradient tracking**:
  - Для тонкой настройки предварительно обученой сетки: `finetuning a pretrained network`. То есть помечаем некоторые параметры (веса) нейронки как **замароженные**, замораживаем веса. Советую почитать об этом здесь: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
  
  - Чтобы ускорить вычисления, когда мы делаем только `forward` прямой проход, поскольку вычисления на тензорах с gradient tracking менее эффективно, чем без него. Например, когда мы обучили модель и просто хотим применить ее к каким-то входным данным.

Подробнее о вычислительных графах
----------------------------

Концептуально, autograd ведет запись данных (тензоров) и всех выполненных операций (вместе с полученными новыми тензорами) в ориентированном ациклическом графе (DAG, directed acyclic graph), состоящем из `Function` объектов. https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function

В графе DAG листья - это входные тензоры, а корни - выходные тензоры. Проследив этот граф от корней до листьев, вы можете автоматически вычислить градиенты, используя цепное правило (chain rule).

При прямом проходе (forward) autograd делает две вещи одновременно:

- запуск запрашиваемой операции чтобы расчитать результирующий тензор,
- поддерживает *функцию градиента* в DAG.

Обратный проход (backward) вызывается в корне DAG. 
``autograd`` тогда:

- вычисляет градиенты из каждого ``.grad_fn``,
- накапливает их в атрибуте ``.grad`` соответствующего тензора,
- по цепному правилу распространяется до листовых тензоров.

<div class="alert alert-info"><h4>Примечание</h4><p>Графы DAG являются динамическими в PyTorch<br><br> Важно отметить, что граф воссоздается с нуля. После каждого вызова ``.backward()``, autograd начинает заполнять новый граф. Это именно то, что позволяет нам использовать операторы потока управления (control flow statements) в вашей модели. Вы можете изменять форму, размер и операции на каждой итерации, если это необходимо.
</p></div>



Дополнительно: Тензорные градиенты и матрица Якоби
--------------------------------------

Во многих случаях у нас есть скалярная функция потерь, и нам нужно вычислить градиент по некоторым параметрам. Однако бывают случаи, когда выходная функция является произвольным тензором. В этом случае PyTorch позволяет вычислить так называемы, а не фактический градиент.
Для вектор функции $\vec{y}=f(\vec{x})$, где 
$\vec{x}=\langle x_1,\dots,x_n\rangle$ и 
$\vec{y}=\langle y_1,\dots,y_m\rangle$,
градиент $\vec{y}$ по отношению к $\vec{x}$ даст **произведение Якоби**:

\begin{align}J=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}
      
      
Вместо того, чтобы вычислять саму матрицу Якоби, PyTorch позволяет вычислить **Произведение Якоби**: $v^T\cdot J$ для заданного входного вектора $v=(v_1 \dots v_m)$. Это достигается вызовом ``backward`` с $v$ в качестве аргумента. 

Размер $v$ должен быть таким же, как размер исходного тензора, относительного которого мы хотим вычислить произведение Якоби:

In [23]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])


In [24]:
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")


Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


Обратите внимание, что когда мы вызываем обратную функцию во второй раз с одним и тем же аргументом, значение аргумента отличается! Почему так? Это происходит потому что при обратном распространении ошибки PyTorch накапливает градиенты, т.е. значение вычисленных градиентов добавляется к свойству `grad` всех листовых узлов вычислительного графа. Если вы хотите вычислить правильные градиенты (без учета накопления), то нужно перед этим обнулить свойство `grad`. В реальном обучении модели нам в этом помогает **оптимизатор**.

<div class="alert alert-info"><h4>Примечание</h4><p>Ранее мы всегда вызывали функцию ``backward()`` без параметров. По сути, это эквивалентно вызову ``backward(torch.tensor(1.0))``, который является полезным способом вычисления градиентов в случае скалярной функции, такой как `loss` во время обучения нейронной сети.
</p></div>




--------------

#### Для подробного ознакомления

`Autograd Mechanics` https://pytorch.org/docs/stable/notes/autograd.html

---